In [1]:
import tensorflow as tf

import numpy as np
import os
import time
import chardet

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
texts = []
dictionary = set()
text = ''

for i in range(15):
    for j in range(49):
        path_to_file = '/home/jupyter/data/g' + str(i) + '/d'+str(j)+'.txt'
            
        file = open(path_to_file, "rb")
        rawdata = file.read()
        file.close()
        result = chardet.detect(rawdata)
        
        try:
            decoded=rawdata.decode(encoding=result['encoding'])
        except:
            None
        else:
            if result['encoding'] == "utf-8":
                formated = decoded.strip()
                formated = formated.replace('\r', '')
                #texts.append(formated)
                text += formated
                dictionary = dictionary.union(set(formated))

In [3]:
vocab = sorted(dictionary)
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [4]:
#int_texts = []
#for txt in texts:
#    int_texts.append(np.array([char2idx[c] for c in txt]))
int_text = np.array([char2idx[c] for c in text])

print("Vocabulary elements", len(vocab))
print("Documents #", len(texts))
print("Text characters #", len(text))

Vocabulary elements 2166
Documents # 0
Text characters # 25678937


In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [6]:
def get_sequences(txt_int):
    # Create training examples / targets
    seq_length = 80
    char_dataset = tf.data.Dataset.from_tensor_slices(txt_int)
    sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
    return sequences.map(split_input_target)

In [7]:
# ============================================== Checkpoint ==============================================
# ============================================== Checkpoint ==============================================
# The maximum length sentence
# datasets = []

dataset = get_sequences(int_text)

#for int_text in int_texts:
#    datasets.append(get_sequences(int_text))

In [8]:
# for input_example, target_example in  datasets[0].take(1):
for input_example, target_example in  dataset.take(1):    
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '<html>\n<head>\n<meta charset="UTF-8"/>\n\n<!--stops zooming-->\n<meta name="viewport'
Target data: 'html>\n<head>\n<meta charset="UTF-8"/>\n\n<!--stops zooming-->\n<meta name="viewport"'


In [9]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset. Maintains a buffer in which it shuffles elements), not all dataset.
BUFFER_SIZE = 10000
#BUFFER_SIZE = 500

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

#for i in range(len(datasets)):
#    datasets[i] = datasets[i].shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [12]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [13]:
optimizer = tf.keras.optimizers.Adam()

In [14]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [15]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints4'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Training step
EPOCHS = 12

for epoch in range(EPOCHS):
    start = time.time()

    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = model.reset_states()

    #for (batch_n, (inp, target)) in enumerate(datasets[0]):
    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)
        
        if batch_n % 200 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 2 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for epoch {} sec\n'.format(time.time() - start))

print("Done")
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 7.679959297180176
Epoch 1 Batch 200 Loss 2.551133632659912
Epoch 1 Batch 400 Loss 2.0129566192626953
Epoch 1 Batch 600 Loss 1.8143341541290283
Epoch 1 Batch 800 Loss 1.466618299484253
Epoch 1 Batch 1000 Loss 1.2992297410964966
Epoch 1 Batch 1200 Loss 0.9799860119819641
Epoch 1 Batch 1400 Loss 1.2368547916412354
Epoch 1 Batch 1600 Loss 1.266495704650879
Epoch 1 Batch 1800 Loss 1.1634080410003662
Epoch 1 Batch 2000 Loss 1.246417760848999
Epoch 1 Batch 2200 Loss 0.9877511858940125
Epoch 1 Batch 2400 Loss 1.0658996105194092
Epoch 1 Loss 1.0278
Time taken for epoch 329.87599420547485 sec

Epoch 2 Batch 0 Loss 6.630316257476807
Epoch 2 Batch 200 Loss 1.2034952640533447
Epoch 2 Batch 400 Loss 1.0775505304336548
Epoch 2 Batch 600 Loss 1.122586727142334
Epoch 2 Batch 800 Loss 1.0678249597549438
Epoch 2 Batch 1000 Loss 1.0100433826446533
Epoch 2 Batch 1200 Loss 0.8078562617301941
Epoch 2 Batch 1400 Loss 0.9690583348274231
Epoch 2 Batch 1600 Loss 1.059451699256897
Epoch 2 Bat

In [16]:
print(tf.train.latest_checkpoint(checkpoint_dir))

#'./training_checkpoints2/ckpt_6'
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

./training_checkpoints4/ckpt_11


In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            554496    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 2166)           2220150   
Total params: 6,712,950
Trainable params: 6,712,950
Non-trainable params: 0
_________________________________________________________________


In [43]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 2000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []
    temperature = 0.8

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [44]:
print("Creating")
print(generate_text(model, start_string=u"<html"))

Creating
<html"></p>
</ class="nam">69 0 112 0 27 3 col-rever-colums-traner="trangrypes/div.proff"},"styleIded":1.699,9,0,0,l, ===========as.tite-49c9u9 4.77186 0-111-13-10-7l-24-2-13 94.7-70-1-1-2.0 zl -->                         <ul>
                                 <div class="fo-xworlay" ta></div>
</div></div></div>
</div></div></div>
</div>
</div>
</div>
</div>
<div class="empont"><a href="/js","data-iage: roode-911la00029"><a href="https://www.wiht.products.ae/2018/00017/04-22-26-15-32-243f-129-712-834-43-313-2.5146-12-had-11-4.0-1.0'></style>
</div>
</div><bar class='trefublogta-fightle-wrapplect-slis.data-ed-tp-content/themontenter"></i>
			
						<div class="fa-f0-20-9 51-1-1 8-4 0-2-48-61-58-120-1 4-2 59-5ec-2122 20 22-327 71.0 2-13-2 2-4 4-17-2.<se-content/pemarthting-height"><il class="ims-453-343-44 -7-1-7-112-2 13-9 63 5 63 min-sitem-colation: 000;sX-483-300 0 5541 248 0 0 84 columnynery-15px 0 1 2 63 23 3z-1-14 4-333-4 8 class="spost_type">
<div class="col-md-620-4"><lia